In [1]:
import json
import pandas as pd
import requests

In [3]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [4]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [23]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [24]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [25]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [26]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [27]:
access_token = get_access_token(client_key, client_secret)

KeyError: 'access_token'

keeping track with calls
27.02 - we ran keys2 unril 1000 


In [11]:
reposted_videos = []
for username in usernames[1001:2002]:
    reposted_videos.append(get_repost_info(username, access_token))

In [15]:
reposted_videos

[{'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 1,
    'id': 7475042232265919786,
    'username': 'charmaine.parsons'},
   {'id': 7474420186079137066,
    'username': 'liladoeslife',
    'favorites_count': 9029},
   {'favorites_count': 113541,
    'id': 7473566114212039958,
    'username': 'alsomiska'},
   {'favorites_count': 17488, 'id': 7475116752926641454, 'username': 'bernie'},
   {'favorites_count': 9135,
    'id': 7474679442145643818,
    'username': 'luisitorio787'},
   {'id': 7474242575562460446,
    'username': 'lookingglass1972',
    'favorites_count': 35},
   {'favorites_count': 22015,
    'id': 7475205577749155115,
    'username': 'edisonquill'},
   {'favorites_count': 22731,
    'id': 7474252612301229317,
    'username': 'movieuniverse.ua'},
   {'favorites_count': 1180,
    'id': 7468445854416833822,
    'username': 'orlandovargas597'},
   {'favorites_count': 2108,
    'id': 7474300782922190122,
    'username': 'lisa_the_lawyer'},
   {'favorit

In [30]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [17]:
combined = {}
for i, username in enumerate(usernames[1001:2002]):
    combined[username] = reposted_videos[i]
    
combined 


{'designs_from_time': {'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 1,
    'id': 7475042232265919786,
    'username': 'charmaine.parsons'},
   {'id': 7474420186079137066,
    'username': 'liladoeslife',
    'favorites_count': 9029},
   {'favorites_count': 113541,
    'id': 7473566114212039958,
    'username': 'alsomiska'},
   {'favorites_count': 17488, 'id': 7475116752926641454, 'username': 'bernie'},
   {'favorites_count': 9135,
    'id': 7474679442145643818,
    'username': 'luisitorio787'},
   {'id': 7474242575562460446,
    'username': 'lookingglass1972',
    'favorites_count': 35},
   {'favorites_count': 22015,
    'id': 7475205577749155115,
    'username': 'edisonquill'},
   {'favorites_count': 22731,
    'id': 7474252612301229317,
    'username': 'movieuniverse.ua'},
   {'favorites_count': 1180,
    'id': 7468445854416833822,
    'username': 'orlandovargas597'},
   {'favorites_count': 2108,
    'id': 7474300782922190122,
    'username': 'lisa_the_l

In [21]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

                username             video_id             creator  favourites
0      designs_from_time  7475042232265919786   charmaine.parsons           1
1      designs_from_time  7474420186079137066        liladoeslife        9029
2      designs_from_time  7473566114212039958           alsomiska      113541
3      designs_from_time  7475116752926641454              bernie       17488
4      designs_from_time  7474679442145643818       luisitorio787        9135
...                  ...                  ...                 ...         ...
11010     straitnochase1  7102217412539354414        mworthofgame         880
11011     straitnochase1  7393106929192537387       gigimichellee       11175
11012     straitnochase1  7236129468530773294              dcheat        3288
11013     straitnochase1  7263955008998214955  backyardccomedyllc           0
11014     straitnochase1  7133995306768796971        darealmikelo      332676

[11015 rows x 4 columns]


In [22]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)